In [43]:
from bs4 import BeautifulSoup
import os
from glob import glob
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
from dask import delayed
import pandas as pd
from distributed import Client
from dask_jobqueue import SLURMCluster
from IPython.display import display
import matplotlib.pyplot as plt
import time
import numpy as np
import pyarrow
from dask.diagnostics import ProgressBar
import time
import csv
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import io
import sys
from dask.diagnostics import ProgressBar
from sklearn.model_selection import GroupShuffleSplit
import nltk
from nltk import word_tokenize
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
csv.field_size_limit(sys.maxsize)


# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = True


if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(
                           # Memory and core limits should be sufficient here
                           memory='64GB', cores=8,

                            # Ensure that Dask uses the correct version of Python on the cluster
                            python='/scratch/work/public/dask/{}/bin/python'.format(dask.__version__),                           
                           
                            # Place the output logs in an accessible location
                            job_extra=['--export=NONE --output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])]
    )

    cluster.submit_command = 'slurm'
    cluster.scale(200) 

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37361 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/home/amr10211/us-court-partisan-bias/dask-worker-space/worker-cv56meh3', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/amr10211/us-court-partisan-bias/dask-worker-space/worker-zm9gy8ig', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/amr10211/us-court-partisan-bias/dask-worker-space/worker-rscxknqz', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/amr10211/us-court-partisan-bias/dask-worker-space/worker-fkiar7c5', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/amr10211/us-court-partisan-bias/dask-worker-space/worker-3977y8rs', purging
distributed.diskutils - INFO - 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37361/status,
Dashboard: http://127.0.0.1:37361/status,Workers: 2
Total threads: 2,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38813,Workers: 2
Dashboard: http://127.0.0.1:37361/status,Total threads: 2
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:45461,Total threads: 1
Dashboard: http://127.0.0.1:37191/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:43865,


In [4]:
# #Pull in tokenized data
# train = pd.read_csv('/vast/amr10211/sc-train-tokenized.csv')
# val = pd.read_csv('/vast/amr10211/sc-val-tokenized.csv')
# test = pd.read_csv('/vast/amr10211/sc-test-tokenized.csv')

# X_train = train['decision_text_tokens']
# X_val = val['decision_text_tokens']
# X_test = test['decision_text_tokens']

# y_train = train['scdb_decision_direction']
# y_val = val['scdb_decision_direction']
# y_test = test['scdb_decision_direction']


In [7]:
# X = pd.concat([X_train, X_val], axis = 0)
# y = pd.concat([y_train, y_val], axis=0)

In [26]:
train = pd.read_csv('/vast/amr10211/sc-train.csv')
val = pd.read_csv('/vast/amr10211/sc-val.csv')
test = pd.read_csv('/vast/amr10211/sc-test.csv')

X_train = train['decision_text']
X_val = val['decision_text']
X_test = test['decision_text']

y_train = train['scdb_decision_direction']
y_val = val['scdb_decision_direction']
y_test = test['scdb_decision_direction']

X = pd.concat([X_train, X_val], axis=0)
y = pd.concat([y_train, y_val], axis=0)

In [27]:
#Vectorize data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
X_test = vectorizer.transform(X_test)

In [39]:
len(train)

15785

In [40]:
len(val)

5322

In [41]:
model = GradientBoostingClassifier()
start = time.time()
model.fit(X[:1000],y[:1000])
print(time.time() - start)

39.626059770584106


In [36]:
sklearn.metrics.accuracy_score(model.predict(X_test[:100]),y_test[:100])

0.69

In [31]:
model = GradientBoostingClassifier(random_state=7)

param_grid = {
    'n_estimators': [100, 200, 300, 400],  # Number of boosting stages (trees)
    # 'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinking to prevent overfitting
    'max_depth': [3, 4, 5, 6],  # Maximum depth of individual trees
    # 'min_samples_split': [2, 3, 4],  # Minimum number of samples required to split an internal node
    # 'min_samples_leaf': [1, 2, 3],  # Minimum number of samples required to be at a leaf node
    # 'subsample': [0.8, 0.9, 1.0]  # Fraction of samples used for fitting the trees
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10)
grid_search.fit(X, y)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2; 1/1] START n_estimators=100............................................


KeyboardInterrupt: 

In [42]:
model = GradientBoostingClassifier(random_state=7)

param_grid = {
    'n_estimators': [100]#, 200, 300, 400],  # Number of boosting stages (trees)
    # 'max_depth': [3, 4, 5, 6],  # Maximum depth of individual trees
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, scoring='accuracy', verbose=10)
grid_search.fit(X[:100], y[:100])

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_

output_file = "baseline_model.txt"

with open(output_file, 'w') as file:
    file.write("Best Hyperparameters:\n")
    for param, value in best_params.items():
        file.write(f"{param}: {value}\n")
    file.write(f"\nBest Accuracy: {best_accuracy:.4f}\n")

print(f"Best hyperparameters and accuracy written to {output_file}")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2; 1/1] START n_estimators=100............................................
[CV 1/2; 1/1] END .............n_estimators=100;, score=0.500 total time=   7.3s
[CV 2/2; 1/1] START n_estimators=100............................................
[CV 2/2; 1/1] END .............n_estimators=100;, score=0.520 total time=   7.6s
Best Hyperparameters:
Best hyperparameters and accuracy written to baseline_model.txt
